# Technical Factor - Signed On-Balance Volume (SOBV)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = f"""
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
    WHERE date BETWEEN '1990-01-01' AND '{ed}'
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        open,
        volume,
        IF(close > open, volume, -1 * volume) AS SOBV_INC,
        SUM(SOBV_INC) OVER (PARTITION BY instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS SOBV,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(SOBV > m_lag(SOBV, 1), 1, 0) AS TRBY1,
        IF(SOBV < m_lag(SOBV, 1), 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF((SOBV - m_lag(SOBV, 1)) > 0 AND ((SOBV - m_lag(SOBV, 1)) - (m_lag(SOBV, 1) - m_lag(SOBV, 2))) > 0, 1, 0) AS MTBY1,
        IF((SOBV - m_lag(SOBV, 1)) < 0 AND ((SOBV - m_lag(SOBV, 1)) - (m_lag(SOBV, 1) - m_lag(SOBV, 2))) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(SOBV_INC > 0 AND m_lag(SOBV_INC, 1) < 0, 1, 0) AS RVBY1,
        IF(SOBV_INC < 0 AND m_lag(SOBV_INC, 1) > 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(SOBV >= m_max(SOBV, 20) AND m_lag(SOBV, 1) < m_lag(m_max(SOBV, 20), 1), 1, 0) AS BKBY1,
        IF(SOBV <= m_min(SOBV, 20) AND m_lag(SOBV, 1) > m_lag(m_min(SOBV, 20), 1), 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        SOBV_INC,
        SOBV,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
AND date BETWEEN '{sd}' AND '{ed}'
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"instrument":instrument_list}).df()
df

,date,instrument,SOBV_INC,alpha_vpta_SOBV,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-02,600519.SH,-5002870,805857533,0,1,0,1,0,0,0,1
1,2025-01-03,600519.SH,-3262836,802594697,0,1,0,0,0,0,0,0
2,2025-01-06,600519.SH,-4425512,798169185,0,1,0,1,0,0,0,0
3,2025-01-07,600519.SH,-2422118,795747067,0,1,0,0,0,0,0,0
4,2025-01-08,600519.SH,3525821,799272888,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
238,2025-12-25,600519.SH,2338621,783654601,1,0,1,0,1,0,0,0
239,2025-12-26,600519.SH,-1780299,781874302,0,1,0,1,0,1,0,0
240,2025-12-29,600519.SH,-2630810,779243492,0,1,0,1,0,0,0,1
241,2025-12-30,600519.SH,-3379194,775864298,0,1,0,1,0,0,0,0
